In [146]:
import pandas as pd
import numpy as np
from  math import sqrt
df = pd.read_csv('TestingRatings.txt',header=None)
df.columns = ['MovieID','UserID','Rating']

Get a table of means

In [147]:
mean_vote = df[['UserID','Rating']].groupby('UserID').mean()
#mean_vote

Get the unique UserID
For that we need to know the keys of mean_vote

In [148]:
list_of_UserID = mean_vote.index.values
# list_of_UserID is an array([.....],dtype=int64)

In [150]:
w = pd.DataFrame(index = list_of_UserID, columns = list_of_UserID)

In this way we insert data in the dataframe

In [151]:
w.loc[6766, 11462] = 111111111



In this way we check if w has a missing value

In [152]:
np.isnan(w.loc[6766, 11462])

False

In [153]:
np.isnan(w.loc[6766, 6766])

True

## Get data to compute p(a,j)

Now I start developing the fuction predicted_vote(a,j)
For this I have to  use w and mean_vote as a global variables
In addition, we have to extract all the votes for the movie J. 
Count how many votes the movie has
Update the w(a,i) and w (i,a) values
We have to calculate the corresponding k

In [154]:
votes_for_movie = df.groupby('MovieID').get_group(61).drop('MovieID', axis = 1).set_index('UserID')
votes_for_movie

,Rating
UserID,
1945693,4.0
1640271,4.0
38946,4.0


In [155]:
len(votes_for_movie)

3

Now we have to learn how to itereate in a DataFrame


In [156]:
for key,vote in votes_for_movie.iterrows():
    print(key, votes_for_movie.loc[key,'Rating'] )
    #print(vote)# I think vote is a series
    print(vote['Rating'])

1945693 4.0
4.0
1640271 4.0
4.0
38946 4.0
4.0


The first iteration will be to update the w matrix with a fix a

In [157]:
a = 2043109
w.loc[2043109, 38946] = 111111111
for key,vote in votes_for_movie.iterrows():
    if(np.isnan(w.loc[a, key] )):
        #Compute W and update
        print('jojolete')
    else:
        #Retrieve infomation
        print(w.loc[a, key])
        


jojolete
jojolete
111111111


## Get data to compute and update w(a,i)

Here we have to be careful and only get once the subset of votes of a.
We have to get the votes that i gave.
We have to find the interception between both sets ( which movies have votes from a and i) then we will have to itereate over this subset of movies computein the summation

In [242]:
selected_movie = 8
votes_for_movie = df.groupby('MovieID').get_group(selected_movie).drop('MovieID', axis = 1).set_index('UserID')
usersID_for_movie = votes_for_movie.index.values
for user in usersID_for_movie:
    print(user,end = ' ')

573364 2149668 1089184 2465894 534508 992921 595054 1298304 1661600 553787 1309839 727242 1437668 2170930 1780876 9660 2379200 563186 1539617 1656839 2591126 2455107 1079943 2308589 254671 2633545 1589111 320540 1769901 155279 2448620 1922607 1388257 613651 900685 1962892 1588471 152955 1071775 2490049 1029578 1116113 400756 1181035 2596310 1716520 450465 927007 1703380 448155 1598457 807237 1089865 1917585 760129 500322 1403170 1631898 1088920 1216843 394189 1473381 803330 1695799 556100 1557449 2090369 2152930 1335436 1466743 264988 1471593 1288346 867716 2160334 2554933 1483852 1581954 1785017 1265606 1213065 898979 1490775 2076079 637631 1970495 1962300 157806 1939221 964306 903378 517335 2406002 112790 2051123 1297899 

In [225]:
a = 1664010
i = 2439493
votes_of_a = df.groupby('UserID').get_group(a).drop('UserID', axis = 1).set_index('MovieID')
#votes_of_a

In [226]:
votes_of_i = df.groupby('UserID').get_group(i).drop('UserID', axis = 1).set_index('MovieID')
#votes_of_i

Now we will get the intersection of the movies for which a and i voted

In [227]:
movies_of_a = votes_of_a.index.values
movies_of_i = votes_of_i.index.values
common_movies = votes_of_a.index.intersection(votes_of_i.index)

In [228]:
common_movies


Int64Index([2510, 3149], dtype='int64', name='MovieID')

In [229]:
for value in common_movies:
    print(value)

2510
3149


In [238]:
sum_lin = 0
sum_sqr_a = 0
sum_sqr_i = 0
for movie in common_movies:
    d_a =(votes_of_a.loc[movie,'Rating'] - mean_vote.loc[a,'Rating'])
    d_i =(votes_of_i.loc[movie,'Rating'] - mean_vote.loc[i,'Rating'])
    sum_lin += d_a * d_i
    sum_sqr_a += d_a**2
    sum_sqr_i += d_i**2
from  math import sqrt
w_a_i = sum_lin / sqrt(sum_sqr_a * sum_sqr_i)
w.loc[a,i] = w_a_i 
w.loc[i,a] = w_a_i

AttributeError: 'numpy.float64' object has no attribute 'loc'

-0.72307692307692273